In [1]:
# 更新库
! python -m pip install --upgrade pip
! pip install -U albumentations
! pip install timm
! pip install tqdm

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple


In [2]:
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import DataLoader, Dataset
#import moxing as mox 

from tqdm import tqdm 

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
#mox.file.copy('obs://test-c3a5/food-11.zip', '/home/ma-user/work/food-11.zip') # 复制数据到当前路径
#mox.file.copy('obs://test-c3a5/student_model.bin', '/home/ma-user/work/student_model.bin')

In [ ]:
# 解压
!unzip food-11.zip

In [4]:
def readfile(path, label):
    # label 是一個 boolean variable，代表需不需要回傳 y 值
    image_dir = sorted(os.listdir(path))
    x = np.zeros((len(image_dir), 128, 128, 3), dtype=np.uint8)
    y = np.zeros((len(image_dir)), dtype=np.uint8)
    for i, file in enumerate(image_dir):
        img = cv2.imread(os.path.join(path, file))
        x[i, :, :] = cv2.resize(img,(128, 128))
        if label:
          y[i] = int(file.split("_")[0])
    if label:
      return x, y
    else:
      return x

In [6]:
workspace_dir = './food-11'
#workspace_dir='E:\\2022\\Network_Compression\\food-11'
print("Reading data")
test_x = readfile(os.path.join(workspace_dir, "testing"), False)
print("Size of Testing data = {}".format(len(test_x)))

Reading data
Size of Testing data = 3347


In [7]:
test_transform = transforms.Compose([
    transforms.ToPILImage(),                                    
    transforms.ToTensor(),
])

class ImgDataset(Dataset):
    def __init__(self, x, y=None, transform=None):
        self.x = x
        # label is required to be a LongTensor
        self.y = y
        if y is not None:
            self.y = torch.LongTensor(y)
        self.transform = transform
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        X = self.x[index]
        if self.transform is not None:
            X = self.transform(X)
        if self.y is not None:
            Y = self.y[index]
            return X, Y
        else:
            return X

In [13]:
class StudentNet(nn.Module):
    '''
      在这个Net里面，我们会使用Depthwise & Pointwise Convolution Layer來叠model。
      你会发現，将原本的Convolution Layer换成Dw & Pw以后，Accuracy通常不会降很多。

      另外，取名为StudentNet是因為个odel等会要做Knowledge Distillation。
    '''

    def __init__(self, base=16, width_mult=1):
        '''
          Args:
            base: 这个model一开始的channle数量，每过一层都会*2，直到base*16为止。
            width_mult:为了之後的Network Pruning使用，在base*8 chs的Layer上会 * width_mult代表剪枝后的ch数量。        
        '''
        super(StudentNet, self).__init__()
        multiplier = [1, 2, 4, 8, 16, 16, 18, 18]

        # bandwidth: 每一层Layer所使用的ch數量
        bandwidth = [ base * m for m in multiplier]

        # 我们只Pruning第三层以后的Layer
        for i in range(3, 7):
            bandwidth[i] = int(bandwidth[i] * width_mult)

        self.cnn = nn.Sequential(
            # 第一层我们通常不会拆解Convolution Layer。
            # nn.Conv2d(in_chs, out_chs, kernel_size, stride, padding)
            nn.Sequential(
                nn.Conv2d(3, bandwidth[0], 3, 1, 1),  # bandwidth[0]=1*16
                nn.BatchNorm2d(bandwidth[0]),
                nn.Hardswish(),
                nn.MaxPool2d(2, 2, 0),
            ),
            # 接下来每一个Sequential Block都一样，所以我们只讲一个block
            nn.Sequential(
                # Depthwise Convolution 卷积层（x, x, x）
                nn.Conv2d(bandwidth[0], bandwidth[0], 3, 1, 1, groups=bandwidth[0]),
                # Batch Normalization
                nn.BatchNorm2d(bandwidth[0]),
                # 使用ReLU6的原因是因为如果数字太大，会不好压到float16 / or further qunatization，因此才給个限制。
                nn.Hardswish(),
                # Pointwise Convolution
                nn.Conv2d(bandwidth[0], bandwidth[1], 1),
                # 过完Pointwise Convolution不需要再做ReLU，经验上Pointwise + ReLU效果都会变差。
                nn.MaxPool2d(2, 2, 0),
                # 每过完一个Block就Down Sampling
            ),

            nn.Sequential(
                nn.Conv2d(bandwidth[1], bandwidth[1], 3, 1, 1, groups=bandwidth[1]),
                nn.BatchNorm2d(bandwidth[1]),
                nn.Hardswish(),
                nn.Conv2d(bandwidth[1], bandwidth[2], 1),
                nn.MaxPool2d(2, 2, 0),
            ),

            nn.Sequential(
                nn.Conv2d(bandwidth[2], bandwidth[2], 3, 1, 1, groups=bandwidth[2]),
                nn.BatchNorm2d(bandwidth[2]),
                nn.Hardswish(),
                nn.Conv2d(bandwidth[2], bandwidth[3], 1),
                nn.MaxPool2d(2, 2, 0),
            ),

            # 到这个为止是因为图片被Down Sample很多次了，所以就不做MaxPool
            nn.Sequential(
                nn.Conv2d(bandwidth[3], bandwidth[3], 3, 1, 1, groups=bandwidth[3]),
                nn.BatchNorm2d(bandwidth[3]),
                nn.Hardswish(),
                nn.Conv2d(bandwidth[3], bandwidth[4], 1),
            ),

            nn.Sequential(
                nn.Conv2d(bandwidth[4], bandwidth[4], 3, 1, 1, groups=bandwidth[4]),
                nn.BatchNorm2d(bandwidth[4]),
                nn.Hardswish(),
                nn.Conv2d(bandwidth[4], bandwidth[5], 1),
            ),

            nn.Sequential(
                nn.Conv2d(bandwidth[5], bandwidth[5], 3, 1, 1, groups=bandwidth[5]),
                nn.BatchNorm2d(bandwidth[5]),
                nn.Hardswish(),
                nn.Conv2d(bandwidth[5], bandwidth[6], 1),
            ),

            nn.Sequential(
                nn.Conv2d(bandwidth[6], bandwidth[6], 3, 1, 1, groups=bandwidth[6]),
                nn.BatchNorm2d(bandwidth[6]),
                nn.Hardswish(),
                nn.Conv2d(bandwidth[6], bandwidth[7], 1),
            ),

            # 这边我们用Global Average Pooling。
            # 如果输入图片大小不一样的话，就会因为Global Average Pooling压成一样的形狀，这样子接下来做FC就不会对不起来。
            nn.AdaptiveAvgPool2d((1, 1)),
        )
        self.fc = nn.Sequential(
            # 这边我们直接Project到11维输出答案。  数据集food种类为11
            nn.Linear(bandwidth[7], 11),
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)


In [ ]:
batch_size = 32
test_set = ImgDataset(test_x, transform=test_transform)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
student_net = StudentNet(base=16).cuda()

student_net.load_state_dict(torch.load('student_model.bin'))

In [9]:
student_net.eval()
predictions = []
for batch in tqdm(test_loader):
    imgs = batch
    with torch.no_grad():
        logits = student_net(imgs.to(device))
    predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

  0%|          | 0/105 [00:00<?, ?it/s]C:\Users\CAFE\AppData\Roaming\Python\Python39\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 105/105 [00:07<00:00, 14.20it/s]


In [10]:
with open("predict.csv", 'w') as f:
    f.write('Id,Category\n')
    for i, y in  enumerate(predictions):
        f.write('{},{}\n'.format(i, y))

In [13]:
import moxing as mox 
# 修改为自己的桶目录
mox.file.copy( '/home/ma-user/work/predict.csv','obs://test-c3a5/predict.csv') 